In [2]:
using LinearAlgebra, SpecialFunctions, SimplexQuad
import GaussQuadrature.legendre
function GL(m; a=0, b=1) 
    # m points Gauss-Legendre quadrature for [a, b]
    xi, wi = legendre(m)
    return map( t->((a+b)/2 + t*(b-a)/2), xi ), (b-a) * wi / 2
end
function K_quadrature(K, α, s; m=50) 
    # Creates K matrix according to m quadrature points
    # Method proposed in (Bornemann, 2009)
    x, w = GL(m; a=0, b=last(s))
    w_sqrt = sqrt.(w)
    return (w_sqrt*w_sqrt') .* map(t->K(α,s,t...), [(xi,xj) for xi in x, xj in x]) 
end

"""
Computing the Bessel Kernel
- For x=y case, see https://arxiv.org/pdf/hep-th/9304063.pdf Eq (1.2b).
- For x!=y case, see page 6 of the above paper.
- For x!=y case, alternative but identical representation in https://arxiv.org/pdf/2206.09411.pdf Eq (25)
which is ( Ja(α, √x) * √y * Ja(α-1, √y) - √x * Ja(α-1, √x) * Ja(α, √y) ) / ( 2 * (x - y) )

HEpdf  : pdf of the smallest eigenvalue, hard-edge
HE2pdf : pdf of the second smallest eigenvalue, hard-edge
"""
Ja = besselj
K_Bess(α, x, y) = x==y ? ( Ja(α, √x)^2 - Ja(α+1, √x) * Ja(α-1, √x) ) / 4 :
                                 ( Ja(α+1, √x) * √x * Ja(α, √y) - √y * Ja(α+1, √y) * Ja(α, √x) ) / ( 2 * (x - y) )
K_Bessp(α, s, x, y) = K_Bess(α, x, y) - K_Bess(α, s, x) * K_Bess(α, s, y) / K_Bess(α, s, s)
HEpdf(α, s; m=50) = K_Bess(α, s, s) * det(I-K_quadrature(K_Bessp, α, s; m=m)) 
function HE2pdf(α, s; m=50)
    K = K_quadrature(K_Bessp, α, s; m=m)
    L = (I-K)\K
    return K_Bess(α,s,s)*tr(L)/det(I+L)
end
function compute_moments(α, m; xmax=200, mint=50) 
    # Computes the first four moments of λ1, λ2
    # Mean, Variance, Skewness, Excess kurtosis
    xs_gl, ws_gl = GL(m; a=0, b=xmax)
    f1_gl, f2_gl = HEpdf.(α, xs_gl; m=mint), HE2pdf.(α, xs_gl;m=mint)
    Eλ1 = sum(f1_gl .* ws_gl .* xs_gl)
    Eλ2 = sum(f2_gl .* ws_gl .* xs_gl)
    Vλ1 = sum(f1_gl .* ws_gl .* xs_gl.^2) - Eλ1^2
    Vλ2 = sum(f2_gl .* ws_gl .* xs_gl.^2) - Eλ2^2
    σλ1, σλ2 = sqrt(Vλ1), sqrt(Vλ2)
    Sλ1 = sum(f1_gl .* ws_gl .* ((xs_gl.-Eλ1)./σλ1).^3)
    Sλ2 = sum(f2_gl .* ws_gl .* ((xs_gl.-Eλ2)./σλ2).^3)
    Kλ1 = sum(f1_gl .* ws_gl .* ((xs_gl.-Eλ1)./σλ1).^4)-3
    Kλ2 = sum(f2_gl .* ws_gl .* ((xs_gl.-Eλ2)./σλ2).^4)-3
    return [Eλ1, Vλ1, Sλ1, Kλ1], [Eλ2, Vλ2, Sλ2, Kλ2]
end

### Now some joint pdf related routines
K_Bess_λ1λ2(α, λs, x, y)   = K_Bess(α,x,y) - K_Bess.(α,x,λs)'*(K_Bess.(α,λs,λs')\K_Bess.(α,λs,y))
joint_pdf_λ1λ2(α, x; m=50) = x[1]<x[2] ? det(K_Bess.(α,x,x'))*det(I-K_quadrature(K_Bess_λ1λ2,α,x;m=m)) : 0.0
function compute_Eλ1λ2(α, m; xmax=200, mint=50)
    # Computes Eλ1λ2 using 2D quadrature
    xs_sq, ws_sq = simplexquad(m, [0.0 0; xmax xmax; 0 xmax])
    xs_grid = [xs_sq[i,:] for i in 1:size(xs_sq,1)]
    fs_sq = joint_pdf_λ1λ2.(α,xs_grid; m=mint)
    return sum(ws_sq .* fs_sq .* prod.(xs_grid)) # Eλ1λ2
end
compute_moments(0, 10; xmax=500, mint=10) # pre-compilation
compute_Eλ1λ2(0, 10; xmax=500, mint=20);

In [3]:
# Computing Four moments, α = 0,1,2
m, mint = 40, 30
α = 0;
@time Mλ1_0, Mλ2_0 = compute_moments(α, m; xmax=500, mint=mint)
α = 1;
@time Mλ1_1, Mλ2_1 = compute_moments(α, m; xmax=600, mint=mint)
α = 2;
@time Mλ1_2, Mλ2_2 = compute_moments(α, m; xmax=800, mint=mint)

@show Mλ1_0
@show Mλ2_0
@show Mλ1_1
@show Mλ2_1
@show Mλ1_2
@show Mλ2_2;

  0.264811 seconds (4.38 M allocations: 71.879 MiB, 4.31% gc time, 2.35% compilation time)
  0.233717 seconds (4.38 M allocations: 71.877 MiB, 2.03% gc time)
  0.298118 seconds (4.38 M allocations: 71.877 MiB, 2.92% gc time)
Mλ1_0 = [3.99999999999996, 15.999999999999826, 1.999999999999973, 5.999999999999442]
Mλ2_0 = [24.362714917268054, 140.3673189982726, 0.9241470357144479, 1.2251116173604366]
Mλ1_1 = [10.87312731383609, 55.745139438490014, 1.3203117463993395, 2.541265785799716]
Mλ2_1 = [40.81220349762314, 259.89850968704377, 0.7378007973144822, 0.7649904035805895]
Mλ1_2 = [20.362714917275955, 124.36731899747997, 1.015815254153757, 1.461305925129178]
Mλ2_2 = [60.11281390465883, 416.8514401559851, 0.6226052474806005, 0.5324828701729141]


In [4]:
## Moment computation
m1, m2, mint = 50, 70, 30

α = 0
@time Mλ1_0, Mλ2_0 = compute_moments(α, m1; xmax=500, mint=mint)
@time Eλ1λ2_0 = compute_Eλ1λ2(α, m2; xmax=500, mint=mint)
ρ_0 = (Eλ1λ2_0 - Mλ1_0[1]*Mλ2_0[1])/sqrt(Mλ1_0[2]*Mλ2_0[2])
α = 1
@time Mλ1_1, Mλ2_1 = compute_moments(α, m1; xmax=600, mint=mint)
@time Eλ1λ2_1 = compute_Eλ1λ2(α, m2; xmax=600, mint=mint)
ρ_1 = (Eλ1λ2_1 - Mλ1_1[1]*Mλ2_1[1])/sqrt(Mλ1_1[2]*Mλ2_1[2])
α = 2
@time Mλ1_2, Mλ2_2 = compute_moments(α, m1; xmax=800, mint=mint)
@time Eλ1λ2_2 = compute_Eλ1λ2(α, m2; xmax=800, mint=mint)
ρ_2 = (Eλ1λ2_2 - Mλ1_2[1]*Mλ2_2[1])/sqrt(Mλ1_2[2]*Mλ2_2[2])
@show ρ_0, ρ_1, ρ_2

  0.267068 seconds (5.48 M allocations: 89.845 MiB, 3.88% gc time)
 39.505630 seconds (736.07 M allocations: 16.347 GiB, 3.21% gc time)
  0.329430 seconds (5.48 M allocations: 89.845 MiB, 2.31% gc time)
 47.009734 seconds (736.07 M allocations: 16.347 GiB, 2.75% gc time)
  0.330940 seconds (5.48 M allocations: 89.845 MiB, 2.24% gc time)
 52.195974 seconds (736.07 M allocations: 16.347 GiB, 2.43% gc time)
(ρ_0, ρ_1, ρ_2) = (0.3376190852244968, 0.3917356930215978, 0.4171879154114054)


(0.3376190852244968, 0.3917356930215978, 0.4171879154114054)

In [5]:
## HIGH PRECISION COMPUTATION FOR DIGIT CHECK
m1, m2, mint = 150, 120, 60

α = 0
@time Mλ1_0, Mλ2_0 = compute_moments(α, m1; xmax=1000, mint=mint)
@time Eλ1λ2_0 = compute_Eλ1λ2(α, m2; xmax=1000, mint=mint)
ρ_0 = (Eλ1λ2_0 - Mλ1_0[1]*Mλ2_0[1])/sqrt(Mλ1_0[2]*Mλ2_0[2])
α = 1
@time Mλ1_1, Mλ2_1 = compute_moments(α, m1; xmax=1200, mint=mint)
@time Eλ1λ2_1 = compute_Eλ1λ2(α, m2; xmax=1200, mint=mint)
ρ_1 = (Eλ1λ2_1 - Mλ1_1[1]*Mλ2_1[1])/sqrt(Mλ1_1[2]*Mλ2_1[2])
α = 2
@time Mλ1_2, Mλ2_2 = compute_moments(α, m1; xmax=1400, mint=mint)
@time Eλ1λ2_2 = compute_Eλ1λ2(α, m2; xmax=1400, mint=mint)
ρ_2 = (Eλ1λ2_2 - Mλ1_2[1]*Mλ2_2[1])/sqrt(Mλ1_2[2]*Mλ2_2[2])
@show ρ_0, ρ_1, ρ_2

  3.970780 seconds (65.79 M allocations: 1.051 GiB, 2.21% gc time)
476.477523 seconds (8.65 G allocations: 191.999 GiB, 3.14% gc time)
  3.418667 seconds (65.79 M allocations: 1.051 GiB, 2.55% gc time)
497.773789 seconds (8.65 G allocations: 191.999 GiB, 3.02% gc time)
  4.136196 seconds (65.79 M allocations: 1.051 GiB, 2.13% gc time)
546.012067 seconds (8.65 G allocations: 191.999 GiB, 2.72% gc time)
(ρ_0, ρ_1, ρ_2) = (0.33761908522455175, 0.3917356930213813, 0.4171879154113996)


(0.33761908522455175, 0.3917356930213813, 0.4171879154113996)

## Just having fun with spacings of various $\alpha$ values. Compare with Figure 2 of https://arxiv.org/pdf/0704.1926.pdf

In [ ]:
using Plots
function Asoft(α,dist; mint=30, mf=30, xmin=0, xmax=400)
    x, w = GL(mint; a=xmin, b=xmax)
    return sum(w.*map(t->joint_pdf_λ1λ2(α,[t,t+dist];m=mf), x))
end
t = 0:1:200
plot(t, Asoft.(0,t), label="α=0", title="First eigenvalue spacing, Hard-edge")
for i in 0:1:3
    plot!(t, Asoft.(i,t), label="α=$i")
end
plot!()